<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению. Сессия № 2

Автор материала: программист-исследователь Mail.ru Group, старший преподаватель Факультета Компьютерных Наук ВШЭ Юрий Кашницкий. Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center> Тема 5. Композиции алгоритмов, случайный лес
## <center>Практика. Деревья решений и случайный лес в соревновании Kaggle Inclass по кредитному скорингу

Тут веб-формы для ответов нет, ориентируйтесь на рейтинг [соревнования](https://inclass.kaggle.com/c/beeline-credit-scoring-competition-2), [ссылка](https://www.kaggle.com/t/115237dd8c5e4092a219a0c12bf66fc6) для участия.

Решается задача кредитного скоринга. 

Признаки клиентов банка:
- Age - возраст (вещественный)
- Income - месячный доход (вещественный)
- BalanceToCreditLimit - отношение баланса на кредитной карте к лимиту по кредиту (вещественный)
- DIR - Debt-to-income Ratio (вещественный)
- NumLoans - число заемов и кредитных линий
- NumRealEstateLoans - число ипотек и заемов, связанных с недвижимостью (натуральное число)
- NumDependents - число членов семьи, которых содержит клиент, исключая самого клиента (натуральное число)
- Num30-59Delinquencies - число просрочек выплат по кредиту от 30 до 59 дней (натуральное число)
- Num60-89Delinquencies - число просрочек выплат по кредиту от 60 до 89 дней (натуральное число)
- Delinquent90 - были ли просрочки выплат по кредиту более 90 дней (бинарный) - имеется только в обучающей выборке

In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline

**Загружаем данные.**

In [2]:
train_df = pd.read_csv('../../data/credit_scoring_train.csv', index_col='client_id')
test_df = pd.read_csv('../../data/credit_scoring_test.csv', index_col='client_id')

In [3]:
train_df.head()

,DIR,Age,NumLoans,NumRealEstateLoans,NumDependents,Num30-59Delinquencies,Num60-89Delinquencies,Income,BalanceToCreditLimit,Delinquent90
client_id,,,,,,,,,,
0,0.496289,49.1,13,0,0.0,2,0,5298.360639,0.387028,0
1,0.433567,48.0,9,2,2.0,1,0,6008.056256,0.234679,0
2,2206.731199,55.5,21,1,NaN,1,0,NaN,0.348227,0
3,886.132793,55.3,3,0,0.0,0,0,NaN,0.971930,0
4,0.000000,52.3,1,0,0.0,0,0,2504.613105,1.004350,0


In [4]:
test_df.head()

,DIR,Age,NumLoans,NumRealEstateLoans,NumDependents,Num30-59Delinquencies,Num60-89Delinquencies,Income,BalanceToCreditLimit
client_id,,,,,,,,,
75000,0.488558,39.2,7,2,2.0,0,0,2866.926559,0.369443
75001,0.132810,42.3,8,0,1.0,4,0,4303.412944,1.028329
75002,1784.812905,51.5,5,1,0.0,0,0,NaN,0.081461
75003,0.538571,57.1,30,2,0.0,0,0,7672.294930,0.485850
75004,0.098539,70.1,3,0,0.0,0,0,4507.010360,0.004258


In [5]:
print(f'train shape: {train_df.shape}')
print(f'test shape: {test_df.shape}')

train shape: (75000, 10)
test shape: (75000, 9)


In [6]:
y = train_df['Delinquent90']
train_df.drop('Delinquent90', axis=1, inplace=True)

In [7]:
train_df.head()

,DIR,Age,NumLoans,NumRealEstateLoans,NumDependents,Num30-59Delinquencies,Num60-89Delinquencies,Income,BalanceToCreditLimit
client_id,,,,,,,,,
0,0.496289,49.1,13,0,0.0,2,0,5298.360639,0.387028
1,0.433567,48.0,9,2,2.0,1,0,6008.056256,0.234679
2,2206.731199,55.5,21,1,NaN,1,0,NaN,0.348227
3,886.132793,55.3,3,0,0.0,0,0,NaN,0.971930
4,0.000000,52.3,1,0,0.0,0,0,2504.613105,1.004350


**Посмотрим на число пропусков в каждом признаке.**

In [8]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 75000 entries, 0 to 74999
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   DIR                    75000 non-null  float64
 1   Age                    75000 non-null  float64
 2   NumLoans               75000 non-null  int64  
 3   NumRealEstateLoans     75000 non-null  int64  
 4   NumDependents          73084 non-null  float64
 5   Num30-59Delinquencies  75000 non-null  int64  
 6   Num60-89Delinquencies  75000 non-null  int64  
 7   Income                 60153 non-null  float64
 8   BalanceToCreditLimit   75000 non-null  float64
dtypes: float64(5), int64(4)
memory usage: 5.7 MB


In [9]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 75000 entries, 75000 to 149999
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   DIR                    75000 non-null  float64
 1   Age                    75000 non-null  float64
 2   NumLoans               75000 non-null  int64  
 3   NumRealEstateLoans     75000 non-null  int64  
 4   NumDependents          72992 non-null  float64
 5   Num30-59Delinquencies  75000 non-null  int64  
 6   Num60-89Delinquencies  75000 non-null  int64  
 7   Income                 60116 non-null  float64
 8   BalanceToCreditLimit   75000 non-null  float64
dtypes: float64(5), int64(4)
memory usage: 5.7 MB


**Заменим пропуски медианными значениями.**

In [10]:
train_df['NumDependents'].fillna(train_df['NumDependents'].median(), inplace=True)
train_df['Income'].fillna(train_df['Income'].median(), inplace=True)
test_df['NumDependents'].fillna(test_df['NumDependents'].median(), inplace=True)
test_df['Income'].fillna(test_df['Income'].median(), inplace=True)

/var/folders/b7/w2lxltn5783dr_jn2g6dmngr0000gn/T/ipykernel_90447/4279207062.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_df['NumDependents'].fillna(train_df['NumDependents'].median(), inplace=True)
/var/folders/b7/w2lxltn5783dr_jn2g6dmngr0000gn/T/ipykernel_90447/4279207062.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object o

### Дерево решений без настройки параметров

**Обучите дерево решений максимальной глубины 3, используйте параметр random_state=17 для воспроизводимости результатов.**

In [11]:
first_tree = DecisionTreeClassifier(max_depth=3, random_state=17)
first_tree.fit(train_df, y)

DecisionTreeClassifier(max_depth=3, random_state=17)

**Сделайте прогноз для тестовой выборки.**

In [12]:
first_tree_pred = first_tree.predict(test_df)
first_tree_pred

array([0, 0, 0, ..., 0, 0, 0], shape=(75000,))

**Запишем прогноз в файл.**

In [13]:
def write_to_submission_file(predicted_labels, out_file,
                             target='Delinquent90', index_label="client_id"):
    # turn predictions into data frame and save as csv file
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(75000, 
                                                  predicted_labels.shape[0] + 75000),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [14]:
write_to_submission_file(first_tree_pred, 'credit_scoring_first_tree.csv')

**Если предсказывать вероятности дефолта для клиентов тестовой выборки, результат будет намного лучше.**

In [15]:
first_tree_pred_probs = first_tree.predict_proba(test_df)[:, 1]

In [16]:
write_to_submission_file(first_tree_pred_probs, 'credit_scoring_first_tree_proba.csv')

## Дерево решений с настройкой параметров с помощью GridSearch

**Настройте параметры дерева с помощью `GridSearhCV`, посмотрите на лучшую комбинацию параметров и среднее качество на 5-кратной кросс-валидации. Используйте параметр `random_state=17` (для воспроизводимости результатов), не забывайте про распараллеливание (`n_jobs=-1`).**

In [17]:
tree_params = {'max_depth': list(range(3, 8)), 
               'min_samples_leaf': list(range(5, 13))}

locally_best_tree = GridSearchCV(estimator=first_tree, param_grid=tree_params, n_jobs=-1, cv=5, scoring="roc_auc")
locally_best_tree.fit(train_df, y)

GridSearchCV(cv=5,
             estimator=DecisionTreeClassifier(max_depth=3, random_state=17),
             n_jobs=-1,
             param_grid={'max_depth': [3, 4, 5, 6, 7],
                         'min_samples_leaf': [5, 6, 7, 8, 9, 10, 11, 12]},
             scoring='roc_auc')

In [18]:
locally_best_tree.best_params_, round(locally_best_tree.best_score_, 3)

({'max_depth': 6, 'min_samples_leaf': 12}, np.float64(0.829))

**Сделайте прогноз для тестовой выборки и пошлите решение на Kaggle.**

In [19]:
tuned_tree_pred_probs = locally_best_tree.predict_proba(test_df)[:, 1]

In [20]:
write_to_submission_file(tuned_tree_pred_probs, 'credit_scoring_locally_best_tree_proba.csv')

### Случайный лес без настройки параметров

**Обучите случайный лес из деревьев неограниченной глубины, используйте параметр `random_state=17` для воспроизводимости результатов.**

In [21]:
first_forest = RandomForestClassifier(random_state=17)
first_forest.fit(train_df, y)

RandomForestClassifier(random_state=17)

In [22]:
first_forest_pred = first_forest.predict(test_df)

**Сделайте прогноз для тестовой выборки и пошлите решение на Kaggle.**

In [23]:
write_to_submission_file(first_forest_pred, 'credit_scoring_first_forest_pred.csv')

### Случайный лес c настройкой параметров

**Настройте параметр `max_features` леса с помощью `GridSearhCV`, посмотрите на лучшую комбинацию параметров и среднее качество на 5-кратной кросс-валидации. Используйте параметр random_state=17 (для воспроизводимости результатов), не забывайте про распараллеливание (n_jobs=-1).**

In [24]:
%%time
forest_params = {'max_features': np.linspace(.3, 1, 7)}

locally_best_forest = GridSearchCV(estimator=first_forest, param_grid=forest_params, cv=5, n_jobs=-1, scoring="roc_auc")
locally_best_forest.fit(train_df, y)

CPU times: user 6.28 s, sys: 130 ms, total: 6.41 s
Wall time: 1min 29s


GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=17), n_jobs=-1,
             param_grid={'max_features': array([0.3       , 0.41666667, 0.53333333, 0.65      , 0.76666667,
       0.88333333, 1.        ])},
             scoring='roc_auc')

In [25]:
locally_best_forest.best_params_, round(locally_best_forest.best_score_, 3)

({'max_features': np.float64(0.3)}, np.float64(0.824))

In [26]:
tuned_forest_pred = locally_best_forest.predict_proba(test_df)[:, 1]

In [27]:
write_to_submission_file(tuned_forest_pred, 'credit_scoring_tunded_forest_pred_proba.csv')

**Посмотрите, как настроенный случайный лес оценивает важность признаков по их влиянию на целевой. Представьте результаты в наглядном виде с помощью `DataFrame`.**

In [28]:
pd.DataFrame({'features': train_df.columns, 'importance': locally_best_forest.best_estimator_.feature_importances_}).sort_values('importance') # Ваш код здесь

,features,importance
3,NumRealEstateLoans,0.032858
4,NumDependents,0.038538
5,Num30-59Delinquencies,0.063031
6,Num60-89Delinquencies,0.065612
2,NumLoans,0.093448
7,Income,0.149931
1,Age,0.160547
0,DIR,0.170165
8,BalanceToCreditLimit,0.225870


**Обычно увеличение количества деревьев только улучшает результат. Так что напоследок обучите случайный лес из 300 деревьев с найденными лучшими параметрами. Это может занять несколько минут.**

In [29]:
best_params = locally_best_forest.best_params_
best_params

{'max_features': np.float64(0.3)}

In [30]:
%%time
final_forest = RandomForestClassifier(n_estimators=300, **best_params)
final_forest.fit(train_df, y)
final_forest_pred = final_forest.predict_proba(test_df)[:, 1]
write_to_submission_file(final_forest_pred, 'credit_scoring_final_forest.csv')

CPU times: user 20.4 s, sys: 51.2 ms, total: 20.4 s
Wall time: 20.5 s


**Сделайте посылку на Kaggle.**

In [31]:
from sklearn.model_selection import RandomizedSearchCV

In [35]:
%%time
my_fores = RandomForestClassifier(random_state=17)
forest_params = {
    'max_features': np.linspace(0.3, 0.6, 3),
    'n_estimators': [300,  500],
    'max_depth': [10, 20],
    'min_samples_leaf': [1, 2],
}
locally_best_forest = GridSearchCV(estimator=my_fores, param_grid=forest_params, cv=3, n_jobs=-1, scoring="roc_auc")
locally_best_forest.fit(train_df, y)

CPU times: user 18.9 s, sys: 389 ms, total: 19.3 s
Wall time: 5min 39s


GridSearchCV(cv=3, estimator=RandomForestClassifier(random_state=17), n_jobs=-1,
             param_grid={'max_depth': [10, 20],
                         'max_features': array([0.3 , 0.45, 0.6 ]),
                         'min_samples_leaf': [1, 2],
                         'n_estimators': [300, 500]},
             scoring='roc_auc')

In [38]:
rf = RandomForestClassifier(n_estimators=500, max_depth=10, max_features=np.float64(0.3), min_samples_leaf=2, random_state=17)
rf.fit(train_df, y)
final_forest_pred = rf.predict_proba(test_df)[:, 1]
write_to_submission_file(final_forest_pred, 'credit_scoring_final_forest_proba.csv')